<a href="https://colab.research.google.com/github/remisoulignac/scm_optim_problems/blob/staging/SCM290-GreenDCLocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

#Week 3 Green Location Problem
FastShipCo has has to deliver 20 locations with various demand level and truck constraints. 5 possible locations have been identified for building distribution centers. Only up to 2 DC can be opened. 
Find the best location for the 2 DCs while optimizing total cost including CO2 emissions

In [ ]:
from ortools.sat.python import cp_model
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns

INFINITY = cp_model.INT32_MAX
model = cp_model.CpModel()

#helper functions to deal with arrays
add_constraints_eq=np.vectorize(lambda e,c : model.Add(e==c))
add_constraints_ge=np.vectorize(lambda e,c : model.Add(e>=c))
add_constraints_le=np.vectorize(lambda e,c : model.Add(e<=c))
vararray_to_resultarray=np.vectorize(lambda v:solver.Value(v))
vararray_to_results=np.vectorize(lambda v:f'{v.Name()}={solver.Value(v)}')

#-- INPUTS
MaxOpenedDCs=2

#[X, Y, Demand, TruckSize]
DemandNodes= np.array([
  [79, 6, 107,  26],
  [16, 47, 2,  7.5],
  [60, 37, 134,  7.5],
  [14, 92, 22,  14],
  [100, 41, 71,  26],
  [9, 10, 21,  14],
  [75, 62, 3,  26],
  [59, 56, 65,  7.5],
  [26, 10, 31,  7.5],
  [10, 23, 41,  7.5],
  [26, 51, 3,  14],
  [41, 56, 5,  14],
  [40, 95, 37,  14],
  [0, 100, 78,  14],
  [47, 100, 54,  7.5],
  [9, 13, 6,  28],
  [95, 18, 47,  7.5],
  [74, 82, 20,  28],
  [52, 4, 33,  26],
  [75, 85, 13,  14]
])

#[X, Y]
CandidateDCs= np.array([
  [39, 78],
  [15, 67],
  [88, 26],
  [47, 38],
  [77, 59],
  [15, 36],
  [72, 96],
  [10, 38],
  [39, 70],
  [75, 55],
  [82, 66],
])




#Fuel Comsumption : [Truck Size, FCEmpty in Liter/km, FCFull in Liter/Km]
FC= np.array([
    [7.5, 0.11, 0.134],
    [14, 0.171, 0.228], 
    [26, 0.244, 0.352], 
    [28, 0.255, 0.402]
])

#Shape:CandidateDCs x DemandNodes 
EuclidianDistances=np.array(list(list(math.sqrt((dc[0]-node[0])**2+(dc[1]-node[1])**2) for node in DemandNodes) for dc in CandidateDCs))

#Shape:DemandNodes
FCEmptyForNode=np.array(list(FC[FC[:,0]==truckSize,1][0] for truckSize in DemandNodes[:,3]))
FCFullForNode=np.array(list(FC[FC[:,0]==truckSize,2][0] for truckSize in DemandNodes[:,3]))
LoadFactorForNode=np.array(list(node[2]/node[3] for node in DemandNodes))
CeiledLoadFactorForNode=np.ceil(LoadFactorForNode)
CarbonEmissionFactor=2.6 #Kg of CO2 / Liter
CostPerTonKilometer=1.20 #$ / Ton.km

#- DECISIONS VARIABLES
#Shape : CandidateDCs
IsDCOpened=np.asarray(list(model.NewBoolVar("DC{}".format(dc)) for dc in range(CandidateDCs.shape[0])))
#Shape : DemandNodes x CandidateDCs 
DoesDCServeNode=np.asarray(list(list(model.NewBoolVar("DC{}->Node{}".format(dc,node)) for node in range(DemandNodes.shape[0])) for dc in range(CandidateDCs.shape[0])) )

#-- CONSTRAINTS
# each node is served by at most 1 dc
add_constraints_eq(np.sum(DoesDCServeNode, axis=0), 1)
# max opened DCs
add_constraints_eq(np.sum(IsDCOpened), MaxOpenedDCs)
# a node is served if the dc is opened
for dc in range(CandidateDCs.shape[0]):
  add_constraints_le(DoesDCServeNode[dc,:], IsDCOpened[dc]) 

#-- OBJECTIVE FUNCTION
CostOfCarbon=100
TotalTransportationCost = CostPerTonKilometer*np.sum(np.sum(DemandNodes[:,2]*EuclidianDistances*DoesDCServeNode))
TotalCarbonEmission = CarbonEmissionFactor*np.sum(np.sum(EuclidianDistances*(FCEmptyForNode*CeiledLoadFactorForNode+(FCFullForNode-FCEmptyForNode)*LoadFactorForNode)*DoesDCServeNode))
TotalCost=TotalTransportationCost+CostOfCarbon*TotalCarbonEmission
model.Minimize(TotalCost)

#-- EXECUTION
solver = cp_model.CpSolver()
status = solver.Solve(model)
if status == cp_model.OPTIMAL:
    fig, ax = plt.subplots()
    sns.scatterplot(data=DemandNodes, x=DemandNodes[:,0], y=DemandNodes[:,1], size=DemandNodes[:,2], hue=DemandNodes[:,3],sizes=(40, 400), palette="muted", ax=ax)

    for i in range(CandidateDCs.shape[0]):
      ax.text(x=CandidateDCs[i,0],y=CandidateDCs[i,1],s=i, fontdict=dict(color='red',size=10), bbox=dict(facecolor='yellow' if solver.Value(IsDCOpened[i])==1 else 'black',alpha=0.5))

    plt.show()

    print("Objective=", solver.ObjectiveValue())
    print("TotalTransportationCost", solver.Value(TotalTransportationCost) )
    print("TotalCarbonEmission", solver.Value(TotalCarbonEmission) )
    print("IsDCOpened=\n", vararray_to_results(IsDCOpened))
    print("DoesDCServeNode=\n", vararray_to_results(DoesDCServeNode))
else:
    print('No optimal solution found.')
